In [1]:
import io
import os

import civis
import intake
import intake_civis
import ipywidgets
from civis_aqueduct_utils.github import upload_file_to_github
from ipywidgets.embed import embed_minimal_html

In [2]:
redshift = intake_civis.open_redshift_catalog()
postgres = intake_civis.open_postgres_catalog()

In [3]:
def populate_tab(catalog, tab):
    label = ipywidgets.HTML(value="<h3>Schemas</h3>")
    schema_listing = ipywidgets.Accordion()
    tab.children = (label, schema_listing)
    populate_schemas(catalog, schema_listing)
    
def populate_schemas(catalog, schema_listing):
    children = []
    names = []
    for name, item in catalog.items():        
        assert item.container == "catalog"
        if item.name == "tiger":
            continue
        table_listing = ipywidgets.Accordion()
        populate_tables(item.get(), table_listing)
        names.append(name)
        children.append(table_listing)

    schema_listing.children = tuple(children)
    schema_listing.selected_index = None
    for i, name in enumerate(names):
        schema_listing.set_title(i, name)
        
def populate_tables(catalog, table_listing):
    children = []
    names = []
    for name, item in catalog.items():        
        html = ipywidgets.HTML()
        populate_source(item, html)
        names.append(name)
        children.append(html)

    table_listing.children = tuple(children)
    table_listing.selected_index = None
    for i, name in enumerate(names):
        table_listing.set_title(i, name)

def populate_source(source, html):
    description = source.describe()
    html.value = f"""
    <p><b>Name:</b> {description["name"]}</p>
    <p><b>Description:</b> {description["description"]}</p>
    """
    if description["args"].get("geometry") is not None:
        html.value = html.value + f"""
        <p><b>Geometry:</b>{description["args"]["geometry"]}</p>
        """
    if description["args"].get("crs") is not None:
        html.value = html.value + f"""
        <p><b>Coordinate reference system:</b>{description["args"]["crs"]}</p>
        """

In [4]:
tabs = ipywidgets.Tab()

postgres_tab = ipywidgets.VBox()
redshift_tab = ipywidgets.VBox()
populate_tab(postgres, postgres_tab)
populate_tab(redshift, redshift_tab)

tabs.children = (postgres_tab, redshift_tab)
tabs.set_title(0, "PostgreSQL")
tabs.set_title(1, "Redshift")
tabs

In [5]:
preamble = ipywidgets.HTML("""
<h1>Civis Platform Datasets</h1>

<p>
The Civis Platform contains two primary data warehouses,
Redshift and PostgreSQL. Redshift is a columnar store
which excels at processing large amounts of analytics data quickly.
PostgreSQL is a more traditional relational database which provides
better support for geospatial data and constraints/relations.
</p>
<p>
The below describes the datasets that are currently located in the data warehouses.
Each one is broken up into schemas, which themselves hold related tabular datasets.
</p>
""")

In [6]:
f = io.StringIO()
embed_minimal_html(f, views=[preamble, tabs])
f.seek(0)
content = f.read()

In [7]:
client = civis.APIClient()
REPORT_ID = os.environ.get("TARGET_REPORT")

if REPORT_ID:
    client.reports.patch(REPORT_ID, code_body=content);